In [18]:
#!/usr/bin/env python3
import time
import numpy as np
from redpitaya_scpi import scpi

# ── 1) Connect to the Pitaya SCPI server ─────────────────────────────────────
PITAYA_IP   = '10.165.61.248'   # ← adjust to your Pitaya’s IP or mDNS name
PITAYA_PORT = 5000

try:
    rp = scpi(PITAYA_IP, port=PITAYA_PORT)
    print(f"Connected to Red Pitaya at {PITAYA_IP}:{PITAYA_PORT}")
except Exception as e:
    print(f"ERROR: could not connect to {PITAYA_IP}:{PITAYA_PORT} → {e}")
    exit(1)

# ── 2) Configure acquisition ───────────────────────────────────────────────────
#   • DEC sets the decimation factor: 125 MS/s ÷ 1024 ≈ 122 kS/s
#   • AVG OFF disables on‑FPGA averaging so we can average in Python
rp.tx_txt('ACQ:DEC 1024')    # decimation                            :contentReference[oaicite:0]{index=0}
rp.tx_txt('ACQ:AVG OFF')     # hardware averaging off                :contentReference[oaicite:1]{index=1}

# ── 3) Arm & trigger ──────────────────────────────────────────────────────────
rp.tx_txt('ACQ:START')       # arm acquisition                       :contentReference[oaicite:2]{index=2}
rp.tx_txt('ACQ:TRIG NOW')    # immediate trigger                     :contentReference[oaicite:3]{index=3}

# wait until trigger‑done (“TD”)
while True:
    rp.tx_txt('ACQ:TRIG:STAT?')
    if rp.rx_txt().strip() == 'TD':
        break                      # buffer is now ready                  :contentReference[oaicite:4]{index=4}

# ── 4) Stream for 5 seconds ────────────────────────────────────────────────────
DURATION_SEC = 5.0
SAMPLES_PER_READ = 1000   # fetch 1000 points per channel each loop

data_ch1 = []
data_ch2 = []
t_start = time.time()

while (time.time() - t_start) < DURATION_SEC:
    # fetch latest decimated points from each channel
    block1 = rp.acq_data(1, num_samples=SAMPLES_PER_READ, lat=True, convert=True)
    block2 = rp.acq_data(2, num_samples=SAMPLES_PER_READ, lat=True, convert=True)
    data_ch1.extend(block1)
    data_ch2.extend(block2)

# ── 5) Compute means ───────────────────────────────────────────────────────────
mean_ch1      = np.mean(data_ch1)
mean_ch2      = np.mean(data_ch2)

print(f"\nResults over {DURATION_SEC:.1f} s:")
print(f"  • Mean CH1:      {mean_ch1:.6f} V")
print(f"  • Mean CH2:      {mean_ch2:.6f} V")


Connected to Red Pitaya at 10.165.61.248:5000

Results over 5.0 s:
  • Mean CH1:      -0.000262 V
  • Mean CH2:      0.000904 V
